In [209]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [210]:
import pandas as pd 
import math
from conf import *

pd.set_option('display.float_format', lambda x: '%.3e' % x if float(x) > 5000 or float(x) < 0.01 and float(x) != 0 else ('%d' % x if x.is_integer() else '%.3f' % x))

In [211]:
DIR = '/home/marco/Desktop/HDNAback/hDNA/results/sumofinchandpk51/run_1'

# hyperp = pd.read_csv(f'{DIR}/hyperparameters.csv'); hyperp['values'][:-1].map(float)
data = pd.read_csv(f'{DIR}/simulationdata.csv'); 
data['index'] = data.index
try: data.drop('sequences.1', axis=1, inplace=True)
except KeyError: pass
try: data.drop('Unnamed: 5', axis=1, inplace=True)
except KeyError: pass

In [212]:
import plotly.graph_objects as go

import pandas as pd

In [213]:
varz = []
for i, seq in enumerate(data['sequences']):
    fptsdf = pd.read_csv(f'{DIR}/simulations/{i+1}_{seq}/fpts.csv',index_col=[0])
    fpts = np.array(fptsdf)
    # print(seq)
    # print('mean:', 1/np.mean(fpts))
    rates = [1/float(f) for f in fpts]
    varrates = np.var(rates)
    std = np.sqrt(varrates)
    # print('var :',std)
    varz.append(std/np.sqrt(5000))
data['var'] = pd.Series(varz)

In [214]:
from hdna import valplot, histotime

In [215]:
# data['computational'][0] = data.iloc[0]['experimental']*1.3
# data.iloc[0]
# data['computational'][20] = data.iloc[20]['experimental']*1.1
# data.iloc[0]


In [216]:
plot = valplot(data, 'see', writepath=None, theme='light')

In [217]:
import plotly.express as px
error = [np.log10(mod/exp) for mod, exp in zip(data['computational'], data['experimental'])]
histo = px.histogram(error,nbins=24)
histo.update_xaxes(exponentformat='e')
histo.update_layout(template='presentation',
                    xaxis_title='')
histo.update_traces(opacity=0.8,
                    marker_autocolorscale=False,
                    marker_color='steelblue',
                    marker_line_color='black',
                    marker_line_width=4,
                    showlegend=False,
                    marker_pattern_fgcolor='black')
histo.show()

In [218]:
def pearson(exp, mod):
    avgexp = np.mean(exp)
    avgmod = np.mean(mod)
    sigmaexp = sum([np.power(x-avgexp,2) for x in exp])
    sigmamod = sum([np.power(y-avgmod,2) for y in mod])
    norm = np.sqrt(sigmaexp*sigmamod)
    corr = sum([(x-avgexp)*(y-avgmod) for x, y in zip(exp, mod)])
    return corr/norm

def distance2badness(exp, mod):
    i = 0
    for expe, mode in zip(exp, mod):
        if mode/2 <= expe <= mode*2:
            i += 1
    return i/len(exp)

def rmse(exp, mod):
    N = len(exp)
    err = sum([(np.log10(m/e))**2 for m,e in zip(mod,exp)])
    return np.sqrt(err/N)

In [219]:
pearson(data['experimental'], data['computational'])

0.6831956405492323

In [220]:
distance2badness(data['experimental'], data['computational'])

0.9512195121951219

In [221]:
rmse(data['experimental'], data['computational'])

0.13301914360283046